In [ ]:
import pandas as pd
import atoti as tt
pd.set_option("display.max_columns", None)
import calendar

### Read the sales raw data
- BSSC raw data
- AG raw data

### Read the category cross table to map the customer code

In [ ]:
bssc = pd.read_csv("../SalesData/BSSC_raw_data.csv")
ag = pd.read_csv("../SalesData/ag_raw_data.csv")
category_table = pd.read_csv("../ReferenceTable/category_types.csv")

In [ ]:
# Strip the space
bssc.columns = bssc.columns.str.strip()
ag.columns = ag.columns.str.strip()

In [ ]:
# Filter the data
ag = ag[(ag["S_配銷通路"].isin(["AG", "FT"])) & (ag["S_中計商品代號1"].isin(["LR2", "TR2", "TR3", "TR5"]))]
bssc = bssc[bssc["中計"].str.contains("LSR2|TBR")]

In [ ]:
# Select the columns to keep
ag = ag[["D_實際發貨日期", "S_訂單單號", "S_項次", "S_買方(客戶號碼)", "S_物料編號", "S_物料說明", "S_國別", "S_數量", "S_項目淨值"]]
bssc = bssc[["實際出貨日", "訂單單號", "訂單項次", "買方", "物料", "物料說明", "國別", "銷售數量", "銷貨單價"]]

In [ ]:
ag.columns = bssc.columns

In [ ]:
type_keys = ['實際出貨日', '訂單單號', '訂單項次', '買方', '物料', '物料說明', '國別', '銷售數量', '銷貨單價']
type_values = [str, str, str, str, str, str, str, int, int]
type_dict = dict(zip(type_keys, type_values))

In [ ]:
ag = ag.astype(type_dict)
bssc = bssc.astype(type_dict)

In [ ]:
combined_data = pd.concat([bssc, ag])

In [ ]:
category_dict = dict(zip(category_table["客戶代號"], category_table["大分類"]))
channel_dict = dict(zip(category_table["客戶代號"], category_table["小通路"]))

In [ ]:
combined_data["大分類"] = combined_data["買方"].map(category_dict)
combined_data["小通路"] = combined_data["買方"].map(channel_dict)

In [ ]:
combined_data["Tire Types"] = combined_data["物料"].map(lambda x: "TBR" if x[:3] == "TBR" else "LSR2")

In [ ]:
combined_data["實際出貨日"] = pd.to_datetime(combined_data["實際出貨日"], yearfirst=True)

In [ ]:
combined_data["年份"] = combined_data["實際出貨日"].dt.year
combined_data["月份"] = combined_data["實際出貨日"].dt.month_name()

In [ ]:
combined_data["年份"] = combined_data["年份"].astype(str)

In [ ]:
combined_data["類型"] = "實績"

### 合併實績和OB數據

In [ ]:
ob22 = pd.read_excel("../ReferenceTable/ob22_20220412.xlsx")

In [ ]:
ob22 = ob22.groupby(['Tire Types', '類型', '大分類', '小通路', '年份']).sum().stack().reset_index(name="數量")

In [ ]:
ob22.columns = ['Tire Types', '類型', '大分類', '小通路', '年份', '月份', '數量']

In [ ]:
ob22 = ob22.astype({"年份": str, "數量": int})

In [ ]:
sales22 = combined_data[["Tire Types", "類型", "大分類", "小通路", "年份", "月份", "銷售數量"]]

In [ ]:
sales22.columns = ob22.columns

In [ ]:
sales22 = sales22.groupby(["Tire Types", "類型", "大分類", "小通路", "年份", "月份"])["數量"].sum().reset_index()

In [ ]:
# Combine the data for atoti analysis
atoti = pd.concat([sales22, ob22])

### 建立Atoti session
1. 建立session
2. 建立cube
3. 建立hierarchy, level, measure

In [ ]:
session = tt.create_session(config={"user_content_storage": "./atoti_pages", "port": 12345})

In [ ]:
sales = session.read_pandas(atoti, keys=["Tire Types", "類型", "大分類", "小通路", "年份", "月份"], table_name="Sales")

In [ ]:
cube = session.create_cube(sales)

In [ ]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [ ]:
h["通路別"] = [l["Tire Types"], l["大分類"], l["小通路"]]
h["時間"] = [l["年份"], l["月份"]]

In [ ]:
h["TBR(G)"] = [sales["大分類"], sales["小通路"]]

In [ ]:
h["數據排序"] = [sales["年份"], sales["類型"], sales["月份"]]

In [ ]:
months = [calendar.month_name[i] for i in range(1, 13)]
months

In [ ]:
l["Sales", "數據排序", "月份"].comparator = tt.comparator.first_members('January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December')
l["Sales", "月份", "月份"].comparator = tt.comparator.first_members('January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December')
l["Sales", "時間", "月份"].comparator = tt.comparator.first_members('January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December')

In [ ]:
session.link()

In [ ]:
m

In [ ]:
m["21 sales.SUM"] = tt.where((l["Sales", "類型", "類型"] == "實績") & (l["Sales", "年份", "年份"] == "2021"), tt.agg.sum(sales["數量"]), 0)

In [ ]:
m["22 OB.SUM"] = tt.where((l["Sales", "類型", "類型"] == "OB"), tt.agg.sum(sales["數量"]), 0)

In [ ]:
m["22 SALES.SUM"] = tt.where((l["Sales", "類型", "類型"] == "實績") & (l["Sales", "年份", "年份"] == "2022"), tt.agg.sum(sales["數量"]), 0)